 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'airlines_delay.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-11-25 19:05:08】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\airlines_delay.yaml
【INFO】【2025-11-25 19:05:15】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-11-25 19:05:15】【配置-数据】数据集=airlines_delay_1m, k折=1, 目标列=DepDelay, 正类="1"
【INFO】【2025-11-25 19:05:15】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-25 19:05:15】【配置-基线】LogReg启用=False, RandomForest启用=False, KNN启用=False, XGBoost启用=False
【INFO】【2025-11-25 19:05:15】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)

# 1）画 0/1 标签（延误/不延误）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='延误 vs 未延误比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始 DepDelay 的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是 "DepDelay"
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Airlines 原始数据加载与基本统计完成。')


【INFO】【2025-11-25 19:05:15】【数据加载】文本表格 ../data/airline/airlines_train_regression_200000.csv 已读取，样本数=200000，列数=10
【INFO】【2025-11-25 19:05:15】【目标变换】已按阈值 15.0 生成二分类标签列 label，正类取 > 15.0
【INFO】【2025-11-25 19:05:15】【数据集信息】名称=airlines_delay_1m，样本数=200000，目标列=label，正类比例=15.60%


,DepDelay,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,Origin,Dest,Distance,label
0,-7.0,4.0,26.0,5.0,935.0,1050.0,MQ,MIA,JAX,335.0,0
1,-2.0,2.0,10.0,6.0,740.0,911.0,DL,MLB,ATL,443.0,0
2,40.0,10.0,30.0,4.0,820.0,930.0,MQ,DFW,LIT,304.0,1
3,0.0,1.0,8.0,2.0,1220.0,1355.0,WN,OAK,PDX,543.0,0
4,0.0,5.0,7.0,7.0,1750.0,1835.0,NW,LIT,MEM,130.0,0


用于建模的标签列: label
原始目标列: DepDelay
count    200000.000000
mean          8.192070
std          28.680684
min        -534.000000
25%          -3.000000
50%           0.000000
75%           7.000000
max        1438.000000
Name: DepDelay, dtype: float64
【INFO】【2025-11-25 19:05:15】【步骤2摘要】Airlines 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-11-25 19:05:15】【预处理】连续特征=6个，类别特征=3个
【INFO】【2025-11-25 19:05:16】【预处理】编码后维度=718
【INFO】【2025-11-25 19:05:16】【预处理】编码特征维度=718，样本数=200000
【INFO】【2025-11-25 19:05:16】【步骤3摘要】特征预处理完成：连续=6，类别=3，编码维度=718。


In [5]:
# 步骤4：构建桶树并检查划分
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=df_raw.drop(columns=[cfg['DATA']['target_col']]).columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(df_raw.drop(columns=[cfg['DATA']['target_col']]))
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(lambda s: (s == cfg['DATA']['positive_label']).mean()).values
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')

【INFO】【2025-11-25 19:05:16】【桶树】已为样本生成桶ID，共 1557 个组合


,bucket_id,count,pos_rate
0,L1_UniqueCarrier=WN|L2_Origin=OTHER|L3_CRSDepT...,6064,0.091743
1,L1_UniqueCarrier=WN|L2_Origin=OTHER|L3_CRSDepT...,6010,0.085106
2,L1_UniqueCarrier=WN|L2_Origin=OTHER|L3_CRSDepT...,3349,0.043011
3,L1_UniqueCarrier=DL|L2_Origin=OTHER|L3_CRSDepT...,2944,0.250000
4,L1_UniqueCarrier=DL|L2_Origin=OTHER|L3_CRSDepT...,2781,0.028169


【INFO】【2025-11-25 19:05:33】【步骤4摘要】桶树划分完成，共有 1557 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-11-25 19:05:33】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-11-25 19:05:33】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,threshold_n_samples,n_all,pos_rate_all,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,L1_UniqueCarrier=OTHER|L2_Origin=OTHER|L3_CRSD...,L3,L1_UniqueCarrier=OTHER|L2_Origin=OTHER,4957,1262,0.184991,0.194929,0.30,0.25,0.558637,...,1262,6219,0.187008,5792.0,0.171443,0.0,0.067852,0.514848,1,0.187008
1,L1_UniqueCarrier=OTHER|L2_Origin=OTHER|L3_CRSD...,L3,L1_UniqueCarrier=OTHER|L2_Origin=OTHER,4874,1248,0.100944,0.080128,0.35,0.30,0.235978,...,1248,6122,0.096700,5800.0,0.105345,0.0,0.001379,0.315086,1,0.096700
2,L1_UniqueCarrier=WN|L2_Origin=OTHER|L3_CRSDepT...,L3,L1_UniqueCarrier=WN|L2_Origin=OTHER,4487,1069,0.198128,0.201123,0.30,0.25,0.576707,...,1069,5556,0.198704,2137.0,0.207768,0.0,0.125409,0.637108,1,0.198704
3,L1_UniqueCarrier=WN|L2_Origin=OTHER|L3_CRSDepT...,L3,L1_UniqueCarrier=WN|L2_Origin=OTHER,4489,1057,0.075963,0.062441,0.30,0.25,0.185904,...,1057,5546,0.073386,2136.0,0.089888,0.0,0.004213,0.274345,1,0.073386
4,L1_UniqueCarrier=OTHER|L2_Origin=OTHER,L2,L1_UniqueCarrier=OTHER,2906,772,0.144529,0.117876,0.30,0.25,0.344560,...,772,3678,0.138934,NaN,NaN,NaN,NaN,NaN,1,0.138934


C:\Users\xzq20.HUANLI\AppData\Local\Temp\ipykernel_24784\946468196.py:9: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


【INFO】【2025-11-25 17:32:47】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [ ]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。')

【INFO】【2025-11-25 17:32:47】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'bucket_thresholds_per_fold.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv']
['bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-25 17:32:47】【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。


: 